In [1]:
# import os
# import cv2
# import numpy as np
# import torch
# from torch.utils.data import Dataset, DataLoader
# from torchvision import transforms
# from PIL import Image

In [2]:
# class MultiViewAnomalyDataset(Dataset):
#     def __init__(self, root_dir, scene_name, sequence_length=2, transform=None, train=True, max_sequences=5):
#         self.root_dir = root_dir
#         self.scene_name = scene_name
#         self.sequence_length = sequence_length
#         self.transform = transform or transforms.Compose([
#             transforms.Resize((32, 32)),  # Very small image size
#             transforms.ToTensor(),
#             transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
#         ])
#         self.train = train
#         self.max_sequences = max_sequences
        
#         # Cache for processed frames
#         self.frame_cache = {}
        
#         # Load frame paths and labels
#         self.view_frames = self._load_view_frames()
        
#         # Pre-process and cache all frames
#         self._preprocess_frames()
        
#     def _preprocess_frames(self):
#         print(f"Pre-processing frames for {self.scene_name} {'Train' if self.train else 'Test'} set...")
#         for view_name, frames in self.view_frames.items():
#             self.frame_cache[view_name] = []
#             for frame in frames:
#                 if isinstance(frame, str):
#                     frame = Image.open(frame).convert('RGB')
#                 else:
#                     frame = Image.fromarray(frame)
                
#                 if self.transform:
#                     frame = self.transform(frame)
#                 self.frame_cache[view_name].append(frame)
#             print(f"Processed {len(self.frame_cache[view_name])} frames for {view_name}")
#     def _load_view_frames(self):
#         scene_dir = os.path.join(self.root_dir, self.scene_name)
#         view_frames = {}
        
#         if not os.path.exists(scene_dir):
#             print(f"Warning: Scene directory {scene_dir} not found")
#             return {}
        
#         # Process available views
#         for view_name in ['View_1', 'View_2', 'View_3']:
#             view_path = os.path.join(scene_dir, view_name)
#             if not os.path.exists(view_path):
#                 print(f"Warning: {view_name} not found in {self.scene_name}")
#                 continue
                
#             if self.train:
#                 frames_dir = os.path.join(view_path, 'Train')
#             else:
#                 frames_dir = os.path.join(view_path, 'Test')
            
#             if not os.path.exists(frames_dir):
#                 print(f"Warning: {frames_dir} not found")
#                 continue
                
#             frame_paths = sorted([
#                 os.path.join(frames_dir, f) for f in os.listdir(frames_dir)
#                 if f.endswith(('.jpg', '.png', '.avi'))
#             ])
            
#             if frame_paths:
#                 if frame_paths[0].endswith('.avi'):
#                     frame_paths = self._extract_frames_from_video(frame_paths[0], stride=32)
                
#                 max_frames = self.max_sequences * self.sequence_length
#                 view_frames[view_name] = frame_paths[:max_frames]
        
#         # Check if we have at least one view
#         if not view_frames:
#             print(f"Warning: No valid views found for {self.scene_name}")
#             return {}
        
#         return view_frames

#     def _extract_frames_from_video(self, video_path, stride=32):
#         frames = []
#         cap = cv2.VideoCapture(video_path)
#         frame_count = 0
        
#         while cap.isOpened() and len(frames) < self.max_sequences * self.sequence_length:
#             ret, frame = cap.read()
#             if not ret:
#                 break
            
#             if frame_count % stride == 0:
#                 frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#                 frames.append(frame)
            
#             frame_count += 1
            
#         cap.release()
#         return frames
#     def __len__(self):
#         if not self.view_frames:
#             return 0
#         min_frames = min(len(frames) for frames in self.view_frames.values())
#         return max(0, min_frames - self.sequence_length + 1)
    
#     def __getitem__(self, idx):
#         sequence_tensors = []
#         required_views = ['View_1', 'View_2', 'View_3']  # Define required views
        
#         # Ensure we process views in the same order
#         for view_name in required_views:
#             if view_name not in self.view_frames:
#                 # If view is missing, create a zero tensor of the right size
#                 view_tensor = torch.zeros(3, self.sequence_length, 64, 64)
#             else:
#                 view_sequence = []
#                 for i in range(self.sequence_length):
#                     frame = self.frame_cache[view_name][idx + i]
#                     view_sequence.append(frame)
                
#                 view_tensor = torch.stack(view_sequence)  # (sequence_length, channels, H, W)
#                 view_tensor = view_tensor.permute(1, 0, 2, 3)  # (channels, sequence_length, H, W)
            
#             sequence_tensors.append(view_tensor)
        
#         sequence_tensor = torch.stack(sequence_tensors)  # (num_views, channels, time, H, W)
#         label = torch.tensor([[0.0]] if self.train else [[1.0]], dtype=torch.float32)
        
#         return sequence_tensor, label

# def get_dataloaders(root_dir, batch_size=2, sequence_length=2, num_workers=0):
#     train_datasets = []
#     test_datasets = []
    
#     for scene_name in ['1_Times_Square', '2_Italy']:
#         print(f"\nProcessing scene: {scene_name}")
        
#         # Training data (normal behavior)
#         train_dataset = MultiViewAnomalyDataset(
#             root_dir=root_dir,
#             scene_name=scene_name,
#             sequence_length=sequence_length,
#             train=True,
#             max_sequences=5
#         )
#         if len(train_dataset) > 0:
#             print(f"Added {len(train_dataset)} training sequences from {scene_name}")
#             train_datasets.append(train_dataset)
        
#         # Testing data (abnormal behavior)
#         test_dataset = MultiViewAnomalyDataset(
#             root_dir=root_dir,
#             scene_name=scene_name,
#             sequence_length=sequence_length,
#             train=False,
#             max_sequences=5
#         )
#         if len(test_dataset) > 0:
#             print(f"Added {len(test_dataset)} testing sequences from {scene_name}")
#             test_datasets.append(test_dataset)
    
#     if not train_datasets and not test_datasets:
#         print("No valid datasets found!")
#         return None, None
    
#     # Create data loaders
#     train_loader = None
#     test_loader = None
    
#     if train_datasets:
#         train_dataset = torch.utils.data.ConcatDataset(train_datasets)
#         train_loader = DataLoader(
#             train_dataset,
#             batch_size=batch_size,
#             shuffle=True,
#             num_workers=num_workers,
#             pin_memory=True
#         )
#         print(f"Created training loader with {len(train_dataset)} sequences")
    
#     if test_datasets:
#         test_dataset = torch.utils.data.ConcatDataset(test_datasets)
#         test_loader = DataLoader(
#             test_dataset,
#             batch_size=batch_size,
#             shuffle=False,
#             num_workers=num_workers,
#             pin_memory=True
#         )
#         print(f"Created testing loader with {len(test_dataset)} sequences")
    
#     return train_loader, test_loader

In [1]:
import os
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

class MultiViewAnomalyDataset(Dataset):
    def __init__(self, root_dir, scene_name='1_Times_Square', sequence_length=2, transform=None, train=True, max_sequences=5):
        self.root_dir = root_dir
        self.scene_name = scene_name  # Only using Times Square
        self.sequence_length = sequence_length
        self.transform = transform or transforms.Compose([
            transforms.Resize((32, 32)),  # Even smaller size
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        self.train = train
        self.max_sequences = max_sequences
        self.frame_cache = {}
        
        # Only use first 3 views
        self.required_views = ['View_1', 'View_2', 'View_3']
        self.frames_per_view = {}
        
        # Load and preprocess frames
        self._load_and_preprocess()
    
    def _load_and_preprocess(self):
        scene_dir = os.path.join(self.root_dir, self.scene_name)
        
        for view_name in self.required_views:
            view_path = os.path.join(scene_dir, view_name)
            split_dir = 'Train' if self.train else 'Test'
            frames_dir = os.path.join(view_path, split_dir)
            
            if not os.path.exists(frames_dir):
                continue
                
            # Get frame paths
            frame_paths = sorted([
                os.path.join(frames_dir, f) for f in os.listdir(frames_dir)
                if f.endswith(('.jpg', '.png', '.avi'))
            ])[:10]  # Only use first 10 frames
            
            # Process frames
            processed_frames = []
            for frame_path in frame_paths:
                if frame_path.endswith('.avi'):
                    img = self._extract_frame_from_video(frame_path)
                else:
                    img = Image.open(frame_path).convert('RGB')
                
                if self.transform:
                    img = self.transform(img)
                processed_frames.append(img)
            
            if processed_frames:
                self.frame_cache[view_name] = processed_frames
                self.frames_per_view[view_name] = len(processed_frames)
    
    def _extract_frame_from_video(self, video_path):
        cap = cv2.VideoCapture(video_path)
        ret, frame = cap.read()
        cap.release()
        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            return Image.fromarray(frame)
        return Image.new('RGB', (32, 32), color='black')
    
    def __len__(self):
        if not self.frame_cache:
            return 0
        min_frames = min(len(frames) for frames in self.frame_cache.values())
        return max(0, min_frames - self.sequence_length + 1)
    
    def __getitem__(self, idx):
        sequence_tensors = []
        
        for view_name in self.required_views:
            if view_name in self.frame_cache:
                frames = self.frame_cache[view_name][idx:idx + self.sequence_length]
                view_tensor = torch.stack(frames)
            else:
                # Create dummy tensor if view is missing
                view_tensor = torch.zeros(self.sequence_length, 3, 32, 32)
            
            view_tensor = view_tensor.permute(1, 0, 2, 3)  # (channels, sequence_length, H, W)
            sequence_tensors.append(view_tensor)
        
        sequence_tensor = torch.stack(sequence_tensors)
        label = torch.tensor([[0.0]] if self.train else [[1.0]], dtype=torch.float32)
        
        return sequence_tensor, label

def get_dataloaders(root_dir, batch_size=2, sequence_length=2, num_workers=0):
    # Only use Times Square scene
    train_dataset = MultiViewAnomalyDataset(
        root_dir=root_dir,
        scene_name='1_Times_Square',
        sequence_length=sequence_length,
        train=True,
        max_sequences=5
    )
    
    test_dataset = MultiViewAnomalyDataset(
        root_dir=root_dir,
        scene_name='1_Times_Square',
        sequence_length=sequence_length,
        train=False,
        max_sequences=5
    )
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True
    ) if len(train_dataset) > 0 else None
    
    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True
    ) if len(test_dataset) > 0 else None
    
    return train_loader, test_loader